In [119]:
#Improt Libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D , MaxPooling2D, Flatten , Dense , Dropout
from keras.models import Sequential
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import joblib

In [77]:
# import pathes
train_dir = r"F:\dataset\task1\data\train"
test_dir = r"F:\dataset\task1\data\test"

In [61]:
# set input shape
input_shape = (128,128,3)
batch_size = 32

In [65]:
# data generator
data_generator = ImageDataGenerator(rescale = 1./255)

In [69]:
# train generator and load images 
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size = (input_shape[0] , input_shape[1]),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = False)

Found 448 images belonging to 2 classes.


In [79]:
# load test images 
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size = (input_shape[0] , input_shape[1]),
    batch_size = batch_size,
    class_mode= 'categorical',
    shuffle = False)

Found 64 images belonging to 3 classes.


In [99]:
# Define the  
model = Sequential([
    Conv2D(32,(3,3),input_shape = input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64,(3,3)),
    Activation('relu'),
    MaxPooling2D(pool_size = (2,2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

In [107]:
# Extract features from train images
train_features = model.predict(train_generator)
train_feature = train_features.reshape(train_features.shape[0],-1)
train_labels = train_generator.classes

14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 364ms/step


In [113]:
# Extract features from test image
test_features = model.predict(test_generator)
test_featres = test_features.reshape(test_features.shape[0] , -1)
test_labels = test_generator.classes

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 389ms/step


In [125]:
# Initialize and train the KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(train_features, train_labels)


# Make predictions on the test data
test_predictions = knn_classifier.predict(test_features)


# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Save the Keras CNN model as an .h5 file
model.save('cnn_Feature_Extractor2.h5')


Test Accuracy: 98.44%


In [127]:
# Save the trained KNN model
joblib.dump(knn_classifier, 'knn_classifier.pkl')

['knn_classifier.pkl']

In [ ]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the saved models
cnn_model = load_model('cnn_Feature_Extractor2.h5')
svm_classifier = joblib.load('svm_classifier.pkl')

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for SVM

    # Classify the features with the SVM model
    prediction = svm_classifier.predict(features)
    
    # Interpret the prediction
    class_labels = {0: 'mask', 1: 'without mask'}  # Adjust based on your dataset labels
    result = class_labels[prediction[0]]
    return result

# Test the function with a new image
img_path = 'C:/Users/it shop/Downloads/Dataset/test/without_mask/without_mask_2039.jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")